# Leer los datos

In [10]:
import pandas as pd
import sys
from itertools import combinations
from pprint import pprint

# Aumentamos el límite de recursión porque el backtracking puede ser muy profundo
sys.setrecursionlimit(10000)

ARCHIVO = "datos.csv"
TOMAS = 30
ACTORES = 10

df = pd.read_csv(ARCHIVO)
TOMAS, ACTORES = df.shape
ACTORES = ACTORES - 3

df.head()

,Toma,1,2,3,4,5,6,7,8,9,10,Unnamed: 11,Total
0,1,1,1,1,1,1,0,0,0,0,0,NaN,5
1,2,0,0,1,1,1,0,0,0,0,0,NaN,3
2,3,0,1,0,0,1,0,1,0,0,0,NaN,3
3,4,1,1,0,0,0,0,1,1,0,0,NaN,4
4,5,0,1,0,1,0,0,0,1,0,0,NaN,3


In [11]:
num_actores = ACTORES
num_tomas = 3 # TOMAS

actores = [str(idx) for idx in range(1, num_actores+1)]
tomas = list(range(1, num_tomas+1))
actores_en_toma = dict()

for _, row in df.iterrows():
    t = int(row['Toma'])
    l_actores = [int(col) for col in actores if int(row[col]) == 1]
    actores_en_toma[t] = l_actores

print("Actores:", actores)
print("Tomas:", tomas)
print("Actores en toma:")
pprint(actores_en_toma)


Actores: ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
Tomas: [1, 2, 3]
Actores en toma:
{1: [1, 2, 3, 4, 5],
 2: [3, 4, 5],
 3: [2, 5, 7],
 4: [1, 2, 7, 8],
 5: [2, 4, 8],
 6: [1, 2, 4, 5],
 7: [1, 2, 4, 5],
 8: [1, 2, 6],
 9: [1, 2, 4],
 10: [1, 2, 6, 9],
 11: [1, 2, 3, 5, 8],
 12: [1, 2, 3, 4, 6],
 13: [1, 4, 5],
 14: [1, 3, 6],
 15: [1, 2, 7],
 16: [4, 10],
 17: [1, 3],
 18: [3, 6],
 19: [1, 3],
 20: [1, 3, 4, 5],
 21: [6, 8],
 22: [1, 2, 3, 4],
 23: [1, 3],
 24: [3, 6],
 25: [1, 2, 4, 10],
 26: [1, 3, 5, 9],
 27: [4, 5],
 28: [1, 4],
 29: [1, 5, 6],
 30: [1, 4]}


# Fuerza bruta

In [7]:
def calcular_coste(programacion, actores_por_toma, num_actores=10):
    """
    Dada la programacion (lista de días, cada día lista de tomas),
    calcula el total de actor-días.
    """
    dias_por_actor = [0] * (num_actores + 1)
    for dia in programacion:
        presentes = set()
        for t in dia:
            presentes.update(actores_por_toma[t])
        for a in presentes:
            dias_por_actor[a] += 1
    return sum(dias_por_actor[1:])

def backtrack(pendientes, programacion_actual, actores_por_toma, solucion):

    if not pendientes:
        coste = calcular_coste(programacion_actual, actores_por_toma)
        if coste < solucion["mejor_coste"]:
            print("Mejor encontrado:", coste)
            solucion["mejor_coste"] = coste
            solucion["mejor_programacion"] = [list(d) for d in programacion_actual]
        return

    siguiente = pendientes[0]
    resto = pendientes[1:]

    # Intentar añadir a días existentes
    for i in range(len(programacion_actual)):
        if len(programacion_actual[i]) < 6:
            programacion_actual[i].append(siguiente)
            backtrack(resto, programacion_actual, actores_por_toma, solucion)
            programacion_actual[i].pop()

    # O crear un nuevo día
    if len(programacion_actual) < len(pendientes):
        programacion_actual.append([siguiente])
        backtrack(resto, programacion_actual, actores_por_toma, solucion)
        programacion_actual.pop()


In [ ]:
# Ejecutar fuerza bruta con diferente # de tomas

for n in range(5,30):

    print("N:", n)
    tomas = list(range(1, n+1))    
    resultado = {
        "mejor_coste": 999999,
        "mejor_solucion": []
    }

    backtrack(tomas, [], actores_en_toma, resultado)

    print(f"Coste mínimo (actor-días): {resultado["mejor_coste"]}")
    for día, grupo in enumerate(resultado["mejor_solucion"], start=1):
        print(f"Día {día:2d}: tomas {grupo}")


N: 5
Mejor encontrado: 7
Coste mínimo (actor-días): 7
N: 6
Mejor encontrado: 7
Coste mínimo (actor-días): 7
N: 7
Mejor encontrado: 11
Mejor encontrado: 10
Coste mínimo (actor-días): 10
N: 8
Mejor encontrado: 12
Coste mínimo (actor-días): 12
N: 9
Mejor encontrado: 12
Coste mínimo (actor-días): 12
N: 10
Mejor encontrado: 13
Coste mínimo (actor-días): 13
N: 11
Mejor encontrado: 15
Mejor encontrado: 13
Coste mínimo (actor-días): 13
N: 12
Mejor encontrado: 15
Mejor encontrado: 14
Coste mínimo (actor-días): 14
N: 13
Mejor encontrado: 19
Mejor encontrado: 18
Mejor encontrado: 17
Coste mínimo (actor-días): 17
N: 14
Mejor encontrado: 20
Mejor encontrado: 19
Mejor encontrado: 18
Mejor encontrado: 17
Coste mínimo (actor-días): 17
N: 15
Mejor encontrado: 22
Mejor encontrado: 21
Mejor encontrado: 20
Mejor encontrado: 19
Mejor encontrado: 18


# Optimizar el algoritmo